In [ ]:
import torch

In [ ]:
import datetime
import glob
from functools import lru_cache, partial
from pprint import pprint

import gym_trading_env
import gymnasium as gym
import matplotlib.pyplot as plt
import MultiTrade
import numpy as np
import pandas as pd
import torch
from gym_trading_env.downloader import download
from gym_trading_env.environments import TradingEnv

from gym_trading_env.renderer import Renderer
from IPython.display import display
from ray import train, tune
from tqdm.autonotebook import tqdm
from utils.utils import build_dataset, build_market_image,preprocess_data
from utils.forecast_utils import build_model_get_data,get_dataset_columns

import ray

from statsforecast import StatsForecast

StatsForecast.plot(Y_df, engine='matplotlib')


import warnings
warnings.filterwarnings("ignore")

from neuralforecast.core import NeuralForecast
from neuralforecast.models import NHITS, NBEATS, PatchTST
from neuralforecast.auto import AutoNHITS, AutoLSTM,AutoPatchTST
from neuralforecast.losses.numpy import mae, mse

In [ ]:
COIN_PAIRS=['BTC/USDT','ETH/USDT','SOL/USDT','BNB/USDT','XRP/USDT','ADA/USDT',
            'ETH/BTC','SOL/ETH','BNB/ETH','XRP/ETH',"ADA/ETH",
            'SOL/BTC','SOL/BNB',
            'XRP/BTC','XRP/BNB',
            'ADA/BTC','ADA/BNB',
            ]
target_pair='ETHUSDT'
time_frame="1h"

In [ ]:
data=build_market_image(target_pair='ETH/USDT',time_frame='1h',axis=0)
data
data=data.groupby('symbol').apply(lambda x: x[:pd.Timestamp('2024-01-01')])

# # data.to_csv('data/binance-market-1h.csv')

# data=data.groupby('symbol').apply(lambda x: x[:pd.Timestamp('2024-01-01')])

data=data.reset_index(level=0,drop=True).reset_index()
data=data[data['symbol']=='ETHUSDT'].copy()
data['symbol'].unique()
id_columns=['symbol']
output_columns,feature_columns,drop_columns=get_dataset_columns(data,id_columns=['symbol'])
len(output_columns)+len(data[id_columns[0]].unique())

In [ ]:
data

In [ ]:

data['ds']=data['date_open']
data=data.drop(['date_open','symbol'],axis=1)
front=['y','ds','unique_id']
data=data[front+[c for c in data.columns if c not in front]]
data=data[front]

In [ ]:
data

In [ ]:


fig, ax = plt.subplots(figsize=(12,8))

ax.plot(data['ds'], data['y'])
# ax.scatter(data['ds'], data['y'], marker='o', color='red', label='New article')
ax.set_xlabel('Day')
ax.set_ylabel('Total views')
ax.legend(loc='best')

fig.autofmt_xdate()


plt.tight_layout()

In [ ]:
split=int(len(data)*.7)
print(split)
train = data[:split]
test = data[split:]
future_exog = test[['unique_id','y']]


In [ ]:
horizon = 7
context_length=5*horizon
max_steps=2000
models = [NHITS(h=horizon,
               input_size=context_length,
               max_steps=20000,
               enable_progress_bar=False
               ),
         NBEATS(h=horizon,
               input_size=context_length,
               max_steps=20000,
               enable_progress_bar=False
               
               ),
         PatchTST(h=horizon,
                 input_size=context_length,
                 max_steps=20000,
               enable_progress_bar=False
                 
                 )
                 ]
nf = NeuralForecast(models=models,

                     freq='1h')

In [ ]:
nf.fit(train,
        val_size=horizon,
        time_col='ds',                    
        target_col='y',
        
        )


In [ ]:
nf.save('/Users/sebastiancoombs/Documents/Git/MultiTrader/MultiTrade',overwrite=True)

In [ ]:

Y_pred = nf.predict().reset_index()
preds_df=Y_pred
preds_df['mean_pred']=preds_df[['NHITS', 'NBEATS','PatchTST']].mean(1)
preds_df= preds_df.merge(test, how='left', on=['unique_id', 'ds'])
preds_df=pd.concat([train.tail(48),preds_df])
preds_df

In [ ]:
def plot_preds(plotting_df,col='mean_pred',cut=False):
    plotting_df=plotting_df.reset_index()
    plt.figure(figsize=(10, 5))
    plt.plot(plotting_df['ds'], plotting_df['y'], label='True')

    plot_col='cutoff' if (('cutoff' in plotting_df.columns) and (cut)) else 'ds'
    plt.plot(plotting_df[plot_col], plotting_df[col], label='Forecast')
    # plt.axvline(preds_df['ds'].iloc[horizon], color='black', linestyle='--', label='Train-Test Split')
    plt.xlabel('Timestamp [t]')
    plt.ylabel('Eth_price')
    plt.grid()
    plt.legend()
    


In [ ]:
model=nf.models[0]
model.max_steps
test_df=test.reset_index(drop=True)
model.max_steps,model.step_size,

In [ ]:
plot_size=500
context_length=5*horizon
end=np.random.randint(0,len(test)-plot_size)
end=end if end>0 else 1
start=end-context_length if (end-context_length)>0 else 0
pred_ids=test_df.index[start:end]
df=test_df.loc[pred_ids]
df=df[['ds','unique_id','y']]
df_plot=test_df.iloc[start:end+horizon].copy().reset_index(drop=True)

nf.dataset, nf.uids, nf.last_dates, nf.ds=nf._prepare_fit(df,
                static_df=None, 
                sort_df=None,
                predict_only=False,
                id_col='unique_id', 
                time_col='ds', 
                target_col='y')


preds_array=np.array([model.predict(nf.dataset) for model in nf.models]).squeeze().mean(0)

# preds_array
df_plot['mean_pred']=np.nan
n_values=int(-preds_array.shape[0])
ids=df_plot.index[n_values:]
df_plot.loc[ids,'mean_pred']=[c for c in preds_array]

plot_preds(df_plot)
# df

In [ ]:
nf.dataset, nf.uids, nf.last_dates, nf.ds=nf._prepare_fit(test_df,
                static_df=None, 
                sort_df=None,
                predict_only=False,
                id_col='unique_id', 
                time_col='ds', 
                target_col='y')

In [ ]:
from itertools import Counter

In [ ]:
all_predictions=nf.predict_insample()


In [ ]:
all_predictions['ds']

In [ ]:

forecasts_series=all_predictions.groupby('cutoff').apply(lambda x: x.select_dtypes(np.number).values.flatten())
forecasts_array=[c for c in forecasts_series]

new_df=test_df[test_df['ds'].isin([c for c in forecasts_series.index])]
print(len(test_df),len(forecasts_array),len(new_df))

In [ ]:
for model in nf.models:
    model.enable_progress_bar=False

In [ ]:
nf.dataset, nf.uids, nf.last_dates, nf.ds=nf._prepare_fit(test_df[['ds','unique_id','y']].tail(horizon),
                static_df=None, 
                sort_df=None ,
                predict_only=True,
                id_col='unique_id', 
                time_col='ds', 
                target_col='y')

In [ ]:
i=0
test_df=test.reset_index(drop=True)
test_df.tail(horizon)


In [ ]:
future=nf.make_future_dataframe()
nf.predict()

In [ ]:
test_pred_df=nf.predict_insample().reset_index()
test_pred_df['mean_pred']=test_pred_df[['NHITS', 'NBEATS','PatchTST']].mean(1)
# test_pred_df['mean_pred']=test_pred_df[['mean_pred']].rolling(horizon).mean(1)
test_pred_df


In [ ]:
plot_preds(test_pred_df)
# test_pred_df


In [ ]:
plot_size=500
start=np.random.randint(0,len(test)-plot_size)
plot_df=test_pred_df.iloc[start:start+plot_size]
plot_df=plot_df.sort_values('ds')
# plot_df=plot_df.groupby('cutoff').mean()
plot_df['mean_pred']=plot_df[['mean_pred']].rolling(horizon).mean()
plot_preds(plot_df,cut=True)

In [ ]:
groups=plot_df.groupby('cutoff')
i=0
max_i=10
for name, df in groups:
    i+=1
    display(df)
    if i>max_i:
        break

In [ ]:
plot_df.groupby('cutoff')['mean_pred'].mean()

In [ ]:
preds_df[['mean_pred','y']].plot()

In [ ]:
preds_df = preds_df.round({
    'NHITS': 4,
    'NBEATS': 4,
    'PatchTST': 4,

})

data = {'N-HiTS': [mae(preds_df['NHITS'], preds_df['y']), mse(preds_df['NHITS'], preds_df['y'])],
       'N-BEATS': [mae(preds_df['NBEATS'], preds_df['y']), mse(preds_df['NBEATS'], preds_df['y'])],
       'PatchTST': [mae(preds_df['PatchTST'], preds_df['y']), mse(preds_df['PatchTST'], preds_df['y'])],
       }

metrics_df = pd.DataFrame(data=data)
metrics_df.index = ['mae', 'mse']

metrics_df.style.highlight_min(color='lightgreen', axis=1)